In [1]:
# Launch training job
# We use the Estimator from the SageMaker Python SDK
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost

from read_upload import Read_Upload

bucket = 'abi-datalake'

hyperparams = {
    "max_depth": "3",
    "eta": "0.3",
    "gamma": "4",
    "min_child_weight": "6",
    "subsample": "0.7",
    "objective": "multi:softprob",
    "num_class": "3",
    "num_round": "20",
    "verbosity": "2",
}

instance_type = "ml.m4.xlarge"
output_path = "s3://{}/{}/".format(bucket, "output")
content_type = "libsvm"

role = 'iam_execution_role'

session = Session()
script_path = "script.py"

xgb_script_mode_estimator = XGBoost(
    entry_point=script_path,
    framework_version="1.5-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=role,
    instance_count=2,
    instance_type=instance_type,
    output_path=output_path,
)

train_input = TrainingInput(
    "s3://{}/{}/".format(bucket, "train"), content_type=content_type
)
validation_input = TrainingInput(
    "s3://{}/{}/".format(bucket, "test"), content_type=content_type
)

xgb_script_mode_estimator.fit({"train": train_input, "validation": validation_input})


2022-05-01 16:28:26 Starting - Starting the training job...
2022-05-01 16:28:54 Starting - Preparing the instances for trainingProfilerReport-1651422505: InProgress
.........
2022-05-01 16:30:34 Downloading - Downloading input data......
2022-05-01 16:31:31 Training - Downloading the training image.....[2022-05-01 16:32:20.524 ip-10-0-214-202.ec2.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-05-01:16:32:20:INFO] Imported framework sagemaker_xgboost_container.training
[2022-05-01:16:32:20:INFO] No GPUs detected (normal if no gpus installed)
[2022-05-01:16:32:20:INFO] Invoking user training script.
[2022-05-01:16:32:21:INFO] Module script does not provide a setup.py. 
Generating setup.py
[2022-05-01:16:32:21:INFO] Generating setup.cfg
[2022-05-01:16:32:21:INFO] Generating MANIFEST.in
[2022-05-01:16:32:21:INFO] Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  

In [6]:
predictor = xgb_script_mode_estimator.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge"
)

--------!

In [7]:
payload_path = "s3://{}/{}/{}".format(bucket, "test", "dtest.svm")
rsu = Read_Upload()
payload = rsu.s3_read(payload_path)
# with open('dtest.svm', "rb") as f:
#     payload = f.read()
payload

b'1 0:6.1 1:2.8 2:4.7 3:1.2\n0 0:5.7 1:3.8 2:1.7 3:0.3\n2 0:7.7 1:2.6 2:6.9 3:2.3\n1 0:6 1:2.9 2:4.5 3:1.5\n1 0:6.8 1:2.8 2:4.8 3:1.4\n0 0:5.4 1:3.4 2:1.5 3:0.4\n1 0:5.6 1:2.9 2:3.6 3:1.3\n2 0:6.9 1:3.1 2:5.1 3:2.3\n1 0:6.2 1:2.2 2:4.5 3:1.5\n1 0:5.8 1:2.7 2:3.9 3:1.2\n2 0:6.5 1:3.2 2:5.1 3:2\n0 0:4.8 1:3 2:1.4 3:0.1\n0 0:5.5 1:3.5 2:1.3 3:0.2\n0 0:4.9 1:3.1 2:1.5 3:0.1\n0 0:5.1 1:3.8 2:1.5 3:0.3\n1 0:6.3 1:3.3 2:4.7 3:1.6\n2 0:6.5 1:3 2:5.8 3:2.2\n1 0:5.6 1:2.5 2:3.9 3:1.1\n1 0:5.7 1:2.8 2:4.5 3:1.3\n2 0:6.4 1:2.8 2:5.6 3:2.2\n0 0:4.7 1:3.2 2:1.6 3:0.2\n2 0:6.1 1:3 2:4.9 3:1.8\n0 0:5 1:3.4 2:1.6 3:0.4\n2 0:6.4 1:2.8 2:5.6 3:2.1\n2 0:7.9 1:3.8 2:6.4 3:2\n2 0:6.7 1:3 2:5.2 3:2.3\n2 0:6.7 1:2.5 2:5.8 3:1.8\n2 0:6.8 1:3.2 2:5.9 3:2.3\n0 0:4.8 1:3 2:1.4 3:0.3\n0 0:4.8 1:3.1 2:1.6 3:0.2\n'

In [8]:

runtime_client = session.sagemaker_runtime_client
response = runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name, ContentType="text/libsvm", Body=payload
)
result = response["Body"].read()

print(result)


b'[[0.03427238389849663, 0.9300899505615234, 0.03563760593533516], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.014111118391156197, 0.029557673260569572, 0.956331193447113], [0.03427238389849663, 0.9300899505615234, 0.03563760593533516], [0.03188217431306839, 0.8652240633964539, 0.10289378464221954], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.03427238389849663, 0.9300899505615234, 0.03563760593533516], [0.014111118391156197, 0.029557673260569572, 0.956331193447113], [0.03427238389849663, 0.9300899505615234, 0.03563760593533516], [0.03427238389849663, 0.9300899505615234, 0.03563760593533516], [0.014111118391156197, 0.029557673260569572, 0.956331193447113], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.9395412802696228, 0.03915587067604065, 0.021302785724401474], [0.033882755786180496, 0.919

In [15]:
payload = [6.1, 2.8, 4.7, 1.2]

['6.1', '2.8', '4.7', '1.2']

In [18]:
payload = [6.1, 2.8, 4.7, 1.2]
payload = ",".join([str(a) for a in payload])

In [19]:
payload

'6.1,2.8,4.7,1.2'

In [20]:

#1
runtime_client = session.sagemaker_runtime_client
response = runtime_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name, ContentType="text/csv", Body=payload
)
result = response["Body"].read()

print(result)

b'[[0.03427238389849663, 0.9300899505615234, 0.03563760593533516]]'


In [21]:
predictor.delete_model()
predictor.delete_endpoint()

In [82]:
import sm_model

In [97]:
import importlib
importlib.reload(sm_model)

<module 'sm_model' from '/home/ahenao/Github/ABIChallenge_alejandro&henao/sm_model.py'>

In [98]:
m = sm_model.sagemaker_xgboost()

In [99]:
m.model_deploy()

---------!

In [37]:
r = m.model_predict([6.1, 2, 4, 2])

In [111]:
r

b'[[0.07268423587083817, 0.493028461933136, 0.43428730964660645]]'

In [57]:
p1 = float(''.join(c for c in str(r).split()[0] if (c.isdigit() or c =='.')))
p2 = float(''.join(c for c in str(r).split()[1] if (c.isdigit() or c =='.')))
p3 = float(''.join(c for c in str(r).split()[2] if (c.isdigit() or c =='.')))
p = [p1, p2, p3]
p

[0.07268423587083817, 0.493028461933136, 0.43428730964660645]

In [58]:
m.model_cancel()

In [101]:
import sagemaker

In [107]:
source = 's3://abi-datalake/sagemaker-xgboost-2022-05-01-20-18-16-799/source/sourcedir.tar.gz'
model_data = 's3://abi-datalake/output/sagemaker-xgboost-2022-05-01-20-18-16-799/output/model.tar.gz'
role = 'iam_execution_role'
model = sagemaker.model.Model(
    image_uri='683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.5-1',
    model_data=model_data,
    role=role,
    source_dir = source,
    predictor_cls=sagemaker.xgboost.XGBoostPredictor)
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
    )

---------!

In [108]:
values = [6.1, 2, 4, 2]
values = ",".join([str(a) for a in values])

session = Session()
runtime_client = session.sagemaker_runtime_client
response = runtime_client.invoke_endpoint(
EndpointName=predictor.endpoint_name, ContentType="text/csv", Body=values
)
result = response["Body"].read()
print(result)

b'0.07268423587083817,0.493028461933136,0.43428730964660645\n'
